# Cepstrum Plotting

A continuation of PSD_by_Type notebook. This notebook was started when I was tasked to make spectrograms using the cepstrum of a signal. This should not be done with the OOIPY spectrogram plotting functions but should instead just be hardcoded using the calculated spectrogram functions.

## Imports & Useful Functions

In [1]:
# data
import ooipy
from ooipy.tools import ooiplotlib as ooiplt
import pandas as pd
import numpy as np
#import functions as fn
# plotting
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from matplotlib.backends.backend_agg import FigureCanvasAgg
# IO
from io import BytesIO
import io
import json
from scipy.io import wavfile
import re

from scipy.signal import welch

def cepstrum_maker(S):
    w = int(len(S)/10) 
    time_windows = []
    for i in range(1, 11):
        """if (i == 10):
            window = S[((i-1)*w):]
        else:
            window = S[((i-1)*w):(i*w)]"""
        # we lose out on very last value
        window = S[((i-1)*w):(i*w)]
        time_windows.append(window)

    # to see time_windows output
    for idx, wind in enumerate(time_windows):
        #print(f'window {idx}: {wind}')
        #print(f'Window {idx} shape is {wind.shape}')
        # compute cf
        cf = (np.log(np.abs(np.fft.fft(wind))))
        #if (len(wind) == 12000):
        #    dict[idx].append(cf)
    return np.mean(time_windows, axis=0)
    


def get_cepsPSD_dict(meta_data_CSV_path, location, print_options=False):
    """
    From given csv metadata path and location string, give a dictionary of PSD values using the cepstrum
    """
    rawdf = pd.read_csv('data/'+location+'/'+meta_data_CSV_path, sep=',')
    if (print_options):
        options = ['Cargo', 'Tanker', 'Fishing type']
        df = rawdf[rawdf['ud_group'].isin(options)]
        df = df.reset_index(drop=True)
        #df = rawdf[(df['ud_group']=='Cargo') | (df['ud_group']=='Tanker') | (df['ud_group']=='Fishing type')]
        # EC is the only one with missing ship type and only 4 of them, just ignore those
        # .dropna() followed by .groupby() cleans and groups
        grouped_df = df.dropna(subset=['ud_group', 'VESSEL TYPE']).groupby(['ud_group', 'VESSEL TYPE'])
        #grouped_df.sum().tail(20)
        mean_dict = {}  

        # Create average data
        for group_name, df_group in grouped_df:
            print(group_name)

    options = ['Cargo', 'Tanker', 'Fishing type']
    df = rawdf[rawdf['VESSEL TYPE'].isin(options)]
    df = rawdf.reset_index(drop=True)
    grouped_df = df.dropna(subset=['ud_group', 'VESSEL TYPE']).groupby(['ud_group', 'VESSEL TYPE'])

    # https://stackoverflow.com/questions/27405483/how-to-loop-over-grouped-pandas-dataframe
    PSDs = {}
    for group_name, df_group in grouped_df:
        PSD_freq = []
        PSD_val = []
        dictPSD = { 'frequencies': PSD_freq, 'values': PSD_val}

        # for-loop to compile list of PSD data objects pulled from column
        for row_index, row in df_group.iterrows():
            inst_id = df['instance_id'].iloc[row_index]
            # get pickle files (DL locally? Call some API?)
            # local storage method
            data_path = 'data/' + location + '/'
            wavfilepath = data_path + 'Audio/' + inst_id + '.wav'
            # try-except deals with missing pickles
            try:
                output = wavfile.read(wavfilepath)
                S = output[1]
                total_cf = np.log(np.abs(np.fft.fft(S)))
                total_quefrec = np.fft.ifft(total_cf)
                cf = cepstrum_maker(S)
                icf = np.fft.ifft(cf)
                #print(cf)
                obj = ooipy.hydrophone.basic.HydrophoneData(data=np.real(icf), header=None, node="Axial_Base")
                obj.stats.sampling_rate = 200
                obj.stats.channel = 'HDH'
                obj.compute_psd_welch(L = 256, overlap=0.3)
                #print(obj.stats)
                psd = obj.psd   
                dictPSD['frequencies'].append(psd.freq)
                dictPSD['values'].append(psd.values)
            except FileNotFoundError:
                pass
        PSDs.update({group_name: dictPSD})
    return PSDs

## Spectrogram

### Initial Testing on one Instance

Issue (04/06/2023): inverse cepstrum (quefrency) returns a solid color plot. Cepstrum values seem fine
(04/06/2023) The above doesn't seem like an error. Referencing Jinting's notebook, quefrency values had a few large spikes and lots of small values. This shows up as small similarly-colored lines near the x-axis in the 0th instance.

In [86]:
rawdf = pd.read_csv('data/'+'Axial_Base'+'/'+'AB_isolated_ais_10m_5_20.csv', sep=',')
options = ['Cargo', 'Tanker', 'Fishing type']
df = rawdf[rawdf['VESSEL TYPE'].isin(options)]
df = rawdf.reset_index(drop=True)
grouped_df = df.dropna(subset=['ud_group', 'VESSEL TYPE']).groupby(['ud_group', 'VESSEL TYPE'])

inst_id = df['instance_id'].iloc[0]

wavfilepath = 'data/Axial_Base/' + 'Audio/' + inst_id + '.wav'
try:
    output = wavfile.read(wavfilepath)
    S = output[1]

    w = int(len(S)/10) 
    time_windows = []
    for i in range(1, 11):
        """if (i == 10):
            window = S[((i-1)*w):]
        else:
            window = S[((i-1)*w):(i*w)]"""
        # we lose out on very last value
        window = S[((i-1)*w):(i*w)]
        time_windows.append(window)

    # to see time_windows output
    cf_windows = []
    for idx, wind in enumerate(time_windows):
        #print(f'window {idx}: {wind}')
        #print(f'Window {idx} shape is {wind.shape}')
        # compute cf
        cf = (np.log(np.abs(np.fft.fft(wind))))
        icf = np.real(np.fft.ifft((cf)))
        cf_windows.append(icf[:int(len(cf)/2)]) # only need half, nyquist
    
except FileNotFoundError:
    pass
len(cf_windows[0])


6000

In [94]:
np.asarray((cf_windows)).T.shape

(6000, 10)

In [96]:
Fs = 200
t = np.arange(10 * 60 * Fs /10/2) 

#plt.contourf(np.arange(10), t, np.asarray(np.log(cf_windows)).T)
plt.contourf(np.arange(10), t, np.asarray((cf_windows)).T, cmap=plt.cm.plasma)
plt.ylim((-10))
plt.colorbar()
plt.ylabel('quefrency (s)')
plt.xlabel('Time (min)')
plt.show()

In [74]:
section = icf[:int(len(cf)/2)]
print(section)
print(np.max(section), np.min(section))

[ 7.62546834e+00  1.62688607e-01 -2.54196141e-01 ...  5.23199875e-03
  1.40767965e-03 -1.67275795e-02]
7.625468341244938 -0.25419614094295906


In [88]:
#plt.plot(S[:int(len(S)/10)], label='original signal')
plt.plot(icf, label='icf')
plt.plot(cf, label='cf')

plt.legend()
plt.show()